# Variables
c_in: int = 20 # Number of input channels

seq_len: int = 128 # Length of input sequences

target_window: int = 96 # Length of prediction window, abhängig von config

prediction length = 96 oder auch target window -> das was predicted werden soll!

model uses huber loss for train and validation, test loss remains MSE


In [13]:
class args:
    random_seed=1
    is_training=1
    model_id='exchange'
    model='JTFT'
    data='custom'
    root_path='./dataset/exchange_rate'
    data_path='exchange_rate.csv'
    features='M'
    target='OT'
    freq='h'
    checkpoints='./checkpoints/'
    seq_len=128
    label_len=1
    pred_len=96
    fc_dropout=0.3
    head_dropout=0.3
    patch_len=4
    stride=2
    padding_patch='end'
    revin=1
    affine=0
    subtract_last=0
    decomposition=2
    kernel_size=25
    individual=0
    embed_type=0
    enc_in=8
    dec_in=7
    c_out=7
    d_model=8
    n_heads=2
    e_layers=3
    d_layers=1
    d_ff=12
    moving_avg=25
    factor=1
    distil=True
    dropout=0.3
    embed='timeF'
    activation='gelu'
    output_attention=False
    do_predict=False
    num_workers=2
    itr=1
    train_epochs=100
    batch_size=64
    patience=10
    learning_rate=0.001
    des='Exp'
    loss='mse'
    lradj='constant'
    pct_start=0.3
    use_amp=False
    use_gpu=False
    gpu=0
    use_multi_gpu=False
    devices='0,1,2,3'
    est_flop=False
    n_freq=16
    n_concat_td=32
    d_compress_max=1
    e_layers_tfi=1
    min_epochs=1
    mod_scal_tfi=0.5
    ini_with_low_freq=False
    use_mark=False
    resume_after_epo=0
    sep_time_freq=False
    use_huber_loss=True
    huber_delta=1.0
    b_not_compile=True



In [10]:
import mne
from torch.utils.data import Dataset, DataLoader

from utils_eeg.tuh_eeg_utils import tuh_eeg_load_data_with_annotations, tuh_eeg_apply_TUH_bipolar_montage
from utils_eeg.data_loader import DataLoader as EEGDataLoader


PATH_TRAINING = "/home/Bachelor-Thesis-JLB/TUH_EEG_SEIZ/edf/train"
PATH_TEST = "/home/Bachelor-Thesis-JLB/TUH_EEG_SEIZ/edf/dev"
MONTAGE_ALL = [ # Montage that is included in all EEG Recordings
    'FP1-F7', 
    'F7-T3', 
    'T3-T5', 
    'T5-O1', 
    'FP2-F8', 
    'F8-T4', 
    'T4-T6', 
    'T6-O2',
    'T3-C3', 
    'C3-CZ', 
    'CZ-C4', 
    'C4-T4',
    'FP1-F3', 
    'F3-C3', 
    'C3-P3', 
    'P3-O1', 
    'FP2-F4', 
    'F4-C4', 
    'C4-P4', 
    'P4-O2'
] # len 20


config_dataset = {
    "extensions": [".edf", ".csv_bi"],
    "l_freq": 0.5,
    "h_freq": 50,
    "resample_freq": 50,


}


def preprocess_eeg_data(file_path):
    raw = tuh_eeg_load_data_with_annotations(edf_file=file_path[0], annotations_csv_file=file_path[1])

    # apply bipolar montage
    raw = tuh_eeg_apply_TUH_bipolar_montage(raw, file_path[0], only_return_bipolar=True)

    # only select channels that are included in all recordings
    raw = raw.pick(MONTAGE_ALL)

    # filter data
    raw = raw.filter(l_freq=config_dataset["l_freq"], h_freq=config_dataset["h_freq"], fir_design='firwin', method='fir', verbose=False)

    # resample data
    raw = raw.resample(config_dataset["resample_freq"], npad='auto', verbose=False)

    #  further processing

    return raw



def eeg_data_generator(file_paths, sequence_length, batch_size, pred_len, preprocess_func):
    while True:
        for file_index, file_path in enumerate(file_paths):
            print(f"File: {file_index:>10} of {len(file_paths)}")
            try:
                raw = preprocess_func(file_path)
            except ValueError as e:
                print("ERROR: ", e)
                print("Loading next file")
                continue
            # except e:
            #     print("ERROR: ", e)
            #     print("Loading next file")
            #     continue

            data_array = raw.get_data()


            for start_idx in range(0, len(raw), sequence_length + 1):
                if start_idx + sequence_length > len(raw):
                    break
                data_x, time_x = raw[:, start_idx:start_idx + sequence_length]
                data_y, time_y = raw[:, start_idx + sequence_length:start_idx + sequence_length + pred_len]

                yield data_x, time_x, data_y, time_y


def calc_dataset_length_tuh(file_paths, sequence_length, pred_length, batch_size):
    dataset_length = 0
    print("Calculating dataset length")
    for idx, file_path in enumerate(file_paths):
        if idx % 100 == 0:
            print(f"Processed: {idx:>10} files | current dataset length: {int(dataset_length):>10}", end="\r")
            

        try:
            raw = mne.io.read_raw_edf(file_path[0], preload=False, verbose=False)


        except ValueError as e:
            print("ERROR: ", e)
            print("Loading next file")
            continue

        dataset_length += ((len(raw) / raw.info["sfreq"]) * config_dataset["resample_freq"])\
              // (sequence_length + pred_length + 1)
        

    print(f"Total files processed: {idx:>10} | Dataset length: {int(dataset_length):>10}")
    return int(dataset_length)


class EEGDataset(Dataset):
    def __init__(self, data_path, 
                 preprocess_func, 
                 data_extensions=['.edf', '.csv_bi'], 
                 batch_size=16,
                 sequence_length=128,
                 pred_length=96):
        data_loader = EEGDataLoader(root_path=data_path, extensions=data_extensions)
        self.file_paths = data_loader.file_tuples
        self.preprocess_func = preprocess_func

        self.generator = eeg_data_generator(file_paths=self.file_paths, 
                                            batch_size=batch_size, 
                                            preprocess_func=preprocess_func, 
                                            sequence_length=sequence_length,
                                            pred_len=pred_length)
        
        # calculate dataset length
        self.dataset_length = calc_dataset_length_tuh(file_paths=self.file_paths, 
                                                       batch_size=batch_size, 
                                                       sequence_length=sequence_length, 
                                                       pred_length=pred_length)
        


    def __len__(self):
        # return len(self.file_paths)
        return self.dataset_length

    def __getitem__(self, idx=None):
        return next(self.generator)



In [11]:
dataset_eeg = EEGDataset(data_path=PATH_TRAINING, 
                         preprocess_func=preprocess_eeg_data, 
                         batch_size=16,
                         sequence_length=128,
                         pred_length=96)


Calculating dataset length
Total files processed:       4663 | Dataset length:     72587111


In [16]:
len(dataset_eeg)

725871

In [14]:
import time
for i, data in enumerate(dataset_eeg):
    print(i)
    print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)
    time.sleep(1)

In [54]:
data[0].shape, data[1]

((20, 128),
 array([2.58, 2.6 , 2.62, 2.64, 2.66, 2.68, 2.7 , 2.72, 2.74, 2.76, 2.78,
        2.8 , 2.82, 2.84, 2.86, 2.88, 2.9 , 2.92, 2.94, 2.96, 2.98, 3.  ,
        3.02, 3.04, 3.06, 3.08, 3.1 , 3.12, 3.14, 3.16, 3.18, 3.2 , 3.22,
        3.24, 3.26, 3.28, 3.3 , 3.32, 3.34, 3.36, 3.38, 3.4 , 3.42, 3.44,
        3.46, 3.48, 3.5 , 3.52, 3.54, 3.56, 3.58, 3.6 , 3.62, 3.64, 3.66,
        3.68, 3.7 , 3.72, 3.74, 3.76, 3.78, 3.8 , 3.82, 3.84, 3.86, 3.88,
        3.9 , 3.92, 3.94, 3.96, 3.98, 4.  , 4.02, 4.04, 4.06, 4.08, 4.1 ,
        4.12, 4.14, 4.16, 4.18, 4.2 , 4.22, 4.24, 4.26, 4.28, 4.3 , 4.32,
        4.34, 4.36, 4.38, 4.4 , 4.42, 4.44, 4.46, 4.48, 4.5 , 4.52, 4.54,
        4.56, 4.58, 4.6 , 4.62, 4.64, 4.66, 4.68, 4.7 , 4.72, 4.74, 4.76,
        4.78, 4.8 , 4.82, 4.84, 4.86, 4.88, 4.9 , 4.92, 4.94, 4.96, 4.98,
        5.  , 5.02, 5.04, 5.06, 5.08, 5.1 , 5.12]))

In [15]:
data_loader_eeg = EEGDataLoader(root_path=PATH_TRAINING, extensions=['.edf', '.csv_bi'])

In [17]:
raw = mne.io.read_raw_edf(data_loader_eeg.file_tuples[0][0], preload=False)

Extracting EDF parameters from /home/Bachelor-Thesis-JLB/TUH_EEG_SEIZ/edf/train/aaaaajvs/s002_2011_03_09/03_tcp_ar_a/aaaaajvs_s002_t005.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [27]:
display(raw.info["sfreq"], len(raw))

display(len(raw.get_data()[0]))

display((len(raw) / raw.info["sfreq"]) * config_dataset["resample_freq"]) 

256.0

153856

153856

30050.0

In [ ]:
from exp.exp_main_JTFT import Exp_Main_JTFT
import torch

Exp = Exp_Main_JTFT

setting_base = '{}_{}_sl{}_pl{}_strd{}_freq{}_dm{}_nh{}_el{}_df{}'.format(
        args.model_id,
        args.model,
        args.seq_len,
        args.pred_len,
        args.stride,
        args.n_freq,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_ff)

if args.is_training:
    for ii in range(args.itr):
        setting = setting_base+'_{}{}'.format(args.des,ii)
        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(''.join(setting)))
        exp.train(setting)
        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(''.join(setting)))
        exp.test(setting)
        
        if args.do_predict:
            print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(''.join(setting)))
            exp.predict(setting, True)

        torch.cuda.empty_cache()

else:
    ii = 0 .format(args.des,ii)
    
    exp = Exp(args)  # set experiments
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(''.join(setting)))
    exp.test(setting, test=1)
    torch.cuda.empty_cache()

In [ ]:
from utils_eeg.data_loader import DataLoader as EEG_DataLoader
from torch.utils.data import Dataset, DataLoader

from utils_eeg.tuh_eeg_utils import  tuh_eeg_load_data_with_annotations, tuh_eeg_apply_TUH_bipolar_montage


PATH_TRAINING = "/home/Bachelor-Thesis-JLB/TUH_EEG_SEIZ/edf/train"
PATH_TEST = "/home/Bachelor-Thesis-JLB/TUH_EEG_SEIZ/edf/dev"
MONTAGE_ALL = [ # Montage that is included in all EEG Recordings
    'FP1-F7', 
    'F7-T3', 
    'T3-T5', 
    'T5-O1', 
    'FP2-F8', 
    'F8-T4', 
    'T4-T6', 
    'T6-O2',
    'T3-C3', 
    'C3-CZ', 
    'CZ-C4', 
    'C4-T4',
    'FP1-F3', 
    'F3-C3', 
    'C3-P3', 
    'P3-O1', 
    'FP2-F4', 
    'F4-C4', 
    'C4-P4', 
    'P4-O2'
]


config_dataset = {
    "extensions": [".edf", ".csv_bi"],
    "l_freq": 0.5,
    "h_freq": 50,
    "resample_freq": 50,


}



class EEG_Dataset(Dataset):
    def __init__(self, data_path, args, flag='train'):

        assert flag in ['train', 'test', 'pred'], "flag should be 'train', 'test' or 'pred'"


        # args
        if flag == 'test':
            self.seq_len = args.seq_len
            self.shuffle_flag = False
            self.batch_size = args.batch_size
            self.freq = args.freq
        elif flag == 'pred':
            self.shuffle_flag = False
            self.drop_last = False
            self.batch_size = 1
            self.freq = args.freq

        else: # train
            self.batch_size = args.batch_size
            self.freq = args.freq
            self.shuffle_flag = True
            self.drop_last = True

        # data
        self.data_path = data_path
        self.data_loader = EEG_DataLoader(self.data_path, extensions=config_dataset['extensions'], shuffle=self.shuffle_flag)


        if self.shuffle_flag:
            self.current_eeg_file = self.data_loader.get_next_shuffled()
        else:
            self.current_eeg_file = self.data_loader.get_next()




    def load_current_eeg(self):
        self.current_eeg_data = tuh_eeg_load_data_with_annotations(self.current_eeg_file[0], self.current_eeg_file[1])

        try:
            self.current_eeg_data = tuh_eeg_apply_TUH_bipolar_montage(self.current_eeg_data, self.current_eeg_file[0], onyl_return_bipolar=True)
    	
        except ValueError as e:
            print("ERROR: ", e)
            if self.shuffle_flag:
                self.current_eeg_file = self.data_loader.get_next_shuffled()
            else:
                self.current_eeg_data = self.data_loader.get_next()
            
            self.load_current_eeg()
            return
        
        # create filter
        self.current_eeg_data = self.current_eeg_data.copy().filter(config_dataset["l_freq"], config_dataset['h_freq'], fir_design='firwin', method='fir', verbose=False)

        # resample data
        self.current_eeg_data = self.current_eeg_data.resample(config_dataset["resample_freq"], npad="auto")

        # epoch the data
        epochs, labels = tuh_eeg_create_epochs_with_labels()
        





    def __len__(self):
        return len(self.data_loader.file_tuples)
    
    def __getitem__(self, index):
        return self.data_loader






In [2]:
import torch

# model JTFT
from models.JTFT import Model as JTFT_Model

model = JTFT_Model(args)


d_compress in mapped transformer 1
